# 🌌 N-Body Simulation with Halos

#### Pragun Nepal  
#### June 21, 2025

---

This notebook contains the code for running the N-body Simulation & Halo Formation (with and without axes). The full video of the simulations have been uploaded in my YouTube channel - https://www.youtube.com/@PragunNepal

All the data for this simulation have been obatined from the coupled N-body, FoF-Halo-Finder, and ReionYuga Repositories by Dr. Rajesh Mondal (GitHub : rajeshmondal18 - https://github.com/rajeshmondal18 )

---

## 📚 Table of Contents

1. [Setting up ffmpeg for Jupyter Lab](#Setting_up_ffmpeg_for_Jupyter_Lab)
2. [Code for N-body Simulation with Halos](#Code_for_N-body_Simulation)
3. [Code for N-body Simulation with Halos (with axes)](#Code_for_N-body_Simulation_axes)
4. [Conclusion](#conclusion)

---

## 🔭 <a name="Setting_up_ffmpeg_for_Jupyter_Lab"></a>1. Setting up ffmpeg for Jupyter Lab



```python
import os
import matplotlib as mpl

#Enter the Full Fath of the ffmeg library - It may be different for you
ffmpeg_path = "/opt/homebrew/bin/ffmpeg"

#Prepend its directory to PATH for this session
os.environ["PATH"] = os.path.dirname(ffmpeg_path) + os.pathsep + os.environ["PATH"]

#Tell Matplotlib exactly where to find it
mpl.rcParams["animation.ffmpeg_path"] = ffmpeg_path

#Verification
import shutil
print("→ PATH is now:", os.environ["PATH"].split(os.pathsep)[:3], "…")
print("→ which(ffmpeg):", shutil.which("ffmpeg"))
print("→ rcParams ffmpeg_path:", mpl.rcParams["animation.ffmpeg_path"])
```



If ffmpeg is not installed in your PC. You can do so by visiting the website - https://ffmpeg.org/download.html



---

## ⚙️ <a name="Code_for_N-body_Simulation"></a>2. Code for N-body Simulation with Halos (without axes)

```python
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.spatial import cKDTree

#Redshift value extraction

def extract_z_nbody(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1] + "." + parts[2])

def extract_z_halo(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1])

#Load and sort files

nbody_files = sorted(glob.glob("p_*.txt"), key=extract_z_nbody, reverse=True)
#I am assuming that this notebook is in the same directory as my n-body data files
halo_path = "/Users/pragunnepal/Downloads/500Halos"
#I am assuming that the Halo data files are in a different directory
#In case the path for data files are different, the above approach can be taken
""" 
nbody_path = "your file path here"
nbody_files = sorted(
    glob.glob(os.path.join(nbody_path, "nbody_*.txt")),
    key=extract_redshift_halo,
    reverse=True 
"""
#The above code can also be used - may require modification

halo_files = sorted(glob.glob(os.path.join(halo_path, "halo_*.txt")), key=extract_z_halo, reverse=True)

#Load raw data and redshifts

raw_n = [np.loadtxt(f).reshape(-1,3) for f in nbody_files]
z_n = [extract_z_nbody(f) for f in nbody_files]
raw_h = []
z_h = []
for f in halo_files:
    try:
        data = np.loadtxt(f).reshape(-1,4)
    except:
        data = np.zeros((0,4))
    raw_h.append(data)
    z_h.append(extract_z_halo(f))

#Synchronize halos to N-body redshifts

frames_n, frames_h, frames_z = [], [], []
for data_n, z in zip(raw_n, z_n):
    frames_n.append(data_n)
    frames_z.append(z)
    if z in z_h:
        idx = z_h.index(z)
        frames_h.append(raw_h[idx])
    else:
        all_z = np.array(z_h)
        if z > all_z[0]:
            frames_h.append(raw_h[0])
        elif z < all_z[-1]:
            frames_h.append(raw_h[-1])
        else:
            i = np.where(all_z >= z)[0][-1]
            j = i + 1
            zA, zB = all_z[i], all_z[j]
            A, B = raw_h[i], raw_h[j]
            t = (z - zA) / (zB - zA)
            if A.size == 0 or B.size == 0:
                interp = A if B.size == 0 else B
            else:
                tree = cKDTree(A[:,1:])
                _, idxs = tree.query(B[:,1:], k=1)
                A_matched = A[idxs]
                interp = (1-t)*A_matched + t*B
            frames_h.append(interp)

#Animation parameters

fps = 30
rotation_speed = 0.1
extra_hold = fps * 33
zoom_in, zoom_out = fps * 15, fps * 5
max_zoom = 1.2
motion = len(frames_n)
total_frames = motion + extra_hold

#Compute bounds

samp_n = np.vstack(frames_n[:min(10, motion)])
samp_h = np.vstack([f[:,1:] for f in raw_h if f.size > 0]) if any(f.size > 0 for f in raw_h) else samp_n
samples = np.vstack([samp_n, samp_h])
bounds = [(samples[:,i].min(), samples[:,i].max()) for i in range(3)]
bounds = [(mn - 0.2*(mx-mn), mx + 0.2*(mx-mn)) for mn, mx in bounds]
centers = [0.5*(mn+mx) for mn, mx in bounds]

#Figure setup (no grid, no axes)

fig = plt.figure(figsize=(6,6), dpi=300)
ax = fig.add_subplot(111, projection='3d')
ax.set_facecolor('black')
fig.patch.set_facecolor('black')
ax.axis('off')

#Initial view

elev0, azim0 = 25, 30
ax.view_init(elev0, azim0)

#Scatter objects

scn = ax.scatter([], [], [], s=0.5, alpha=0.1, c='cyan', edgecolors='none')
sch = ax.scatter([], [], [], s=[], alpha=0.7, c='gold', edgecolors='none')
txt = ax.text2D(0.02, 0.95, '', transform=ax.transAxes, color='white')
ax_triad = inset_axes(ax, width='20%', height='20%', loc='lower left')
ax_triad.axis('off')

#Rotation matrix helper

def rotation_matrix(angle_deg):
    theta = np.deg2rad(angle_deg)
    Rz = np.array([[np.cos(theta), -np.sin(theta), 0],
                   [np.sin(theta),  np.cos(theta), 0],
                   [0, 0, 1]])
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(theta), -np.sin(theta)],
                   [0, np.sin(theta),  np.cos(theta)]])
    return Rz @ Rx

#Update function

def update(frame):
    idx = frame if frame < motion else motion - 1
    data_n = frames_n[idx]
    data_h = frames_h[idx]
    zval = frames_z[idx]

    scn._offsets3d = (data_n[:,0], data_n[:,1], data_n[:,2])
    if data_h.size:
        masses = data_h[:,0]
        coords = data_h[:,1:]
        radii = np.cbrt(masses)
        sizes = (radii / radii.max()) * 50
        sch._offsets3d = (coords[:,0], coords[:,1], coords[:,2])
        sch.set_sizes(sizes)
    else:
        sch._offsets3d = ([], [], [])
        sch.set_sizes([])

    txt.set_text(f'z = {zval:.3f}')

    angle = frame * rotation_speed
    ax.view_init(elev0 + angle, azim0 + angle)

    ax_triad.clear()
    ax_triad.axis('off')
    R = rotation_matrix(angle)
    for color, vec, label in zip(['red','green','blue'], [np.array([1,0,0]), np.array([0,1,0]), np.array([0,0,1])], ['X','Y','Z']):
        v2 = R @ vec
        end = np.array([0.5, 0.5]) + 0.3 * v2[:2]
        ax_triad.annotate('', xy=end, xytext=(0.5, 0.5), arrowprops=dict(color=color, arrowstyle='->', alpha=0.6))
        ax_triad.text(end[0], end[1], label, color=color, fontsize=8, alpha=0.6)

    if frame >= motion:
        h = frame - motion
        if h <= zoom_in:
            f = 1 + (max_zoom - 1) * (h / zoom_in)
        else:
            f = max_zoom - (max_zoom - 1) * min((h - zoom_in) / zoom_out, 1)
    else:
        f = 1

    for i, (mn, mx) in enumerate(bounds):
        c = centers[i]
        r = (mx - mn) / 2
        low, high = c - r/f, c + r/f
        if i == 0:
            ax.set_xlim(low, high)
        elif i == 1:
            ax.set_ylim(low, high)
        else:
            ax.set_zlim(low, high)

    return scn, sch, txt

#Animate & save

ani = FuncAnimation(fig, update, frames=total_frames, interval=1000/fps, blit=True)
writer = FFMpegWriter(fps=fps, metadata={'artist':'PragunNepal'}, bitrate=30000)
ani.save('CombinedFinal1.mp4', writer=writer)
print('✅ Saved combined_simulation.mp4')

```
---

## ⚙️ <a name="Code_for_N-body_Simulation_axes"></a>3. Code for N-body Simulation with Halos (with axes)

```python
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.spatial import cKDTree

#Redshift value extraction

def extract_z_nbody(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1] + "." + parts[2])

def extract_z_halo(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1])

#Load and sort files

nbody_files = sorted(glob.glob("p_*.txt"), key=extract_z_nbody, reverse=True)
#I am assuming that this notebook is in the same directory as my n-body data files
halo_path = "/Users/pragunnepal/Downloads/500Halos"
#I am assuming that the Halo data files are in a different directory
#In case the path for data files are different, the above approach can be taken
""" 
nbody_path = "your file path here"
nbody_files = sorted(
    glob.glob(os.path.join(nbody_path, "nbody_*.txt")),
    key=extract_redshift_halo,
    reverse=True 
"""
#The above code can also be used - may require modification

halo_files = sorted(glob.glob(os.path.join(halo_path, "halo_*.txt")), key=extract_z_halo, reverse=True)

#Load raw data and redshifts

raw_n = [np.loadtxt(f).reshape(-1,3) for f in nbody_files]
z_n = [extract_z_nbody(f) for f in nbody_files]
raw_h = []
z_h = []
for f in halo_files:
    try:
        data = np.loadtxt(f).reshape(-1,4)
    except:
        data = np.zeros((0,4))
    raw_h.append(data)
    z_h.append(extract_z_halo(f))

#Synchronize halos to N-body redshifts

frames_n, frames_h, frames_z = [], [], []
for data_n, z in zip(raw_n, z_n):
    # N-body frame
    frames_n.append(data_n)
    frames_z.append(z)
    # find halo interpolation
    # if exact match
    if z in z_h:
        idx = z_h.index(z)
        frames_h.append(raw_h[idx])
    else:
        # find bounding halos
        all_z = np.array(z_h)
        if z > all_z[0]:
            frames_h.append(raw_h[0])
        elif z < all_z[-1]:
            frames_h.append(raw_h[-1])
        else:
            # find surrounding indices i, i+1 with z_h[i] >= z >= z_h[i+1]
            i = np.where(all_z >= z)[0][-1]
            j = i+1
            zA, zB = all_z[i], all_z[j]
            A, B = raw_h[i], raw_h[j]
            t = (z - zA) / (zB - zA)
            # match halos by nearest neighbor
            if A.size==0 or B.size==0:
                interp = A if B.size==0 else B
            else:
                tree = cKDTree(A[:,1:])
                _, idxs = tree.query(B[:,1:], k=1)
                A_matched = A[idxs]
                interp = (1-t)*A_matched + t*B
            frames_h.append(interp)

#Animation parameters

fps = 30
rotation_speed = 0.1
extra_hold = fps * 33
zoom_in, zoom_out = fps * 15, fps * 5
max_zoom = 1.2

motion = len(frames_n)
total_frames = motion + extra_hold

#Compute bounds

samp_n = np.vstack(frames_n[:min(10, motion)])
samp_h = np.vstack([f[:,1:] for f in raw_h if f.size>0]) if any(f.size>0 for f in raw_h) else samp_n
samples = np.vstack([samp_n, samp_h])
bounds = [(samples[:,i].min(), samples[:,i].max()) for i in range(3)]
bounds = [(mn - 0.2*(mx-mn), mx + 0.2*(mx-mn)) for mn,mx in bounds]
centers = [0.5*(mn+mx) for mn,mx in bounds]

#Figure setup

fig = plt.figure(figsize=(6,6), dpi=300)
ax = fig.add_subplot(111, projection='3d')
ax.set_facecolor('black')
fig.patch.set_facecolor('black')
for axis in [ax.xaxis, ax.yaxis, ax.zaxis]:
    axis._axinfo['grid']['color'] = 'purple'
    axis._axinfo['grid']['linewidth'] = 0.1
    axis._axinfo['grid']['alpha'] = 0.001
    axis.pane.set_facecolor('purple')
    axis.pane.set_edgecolor('purple')
    axis.pane.set_alpha(0.001)
ax.tick_params(colors='purple', labelsize=6)
ax.set_box_aspect([1,1,1])
ax.axis('on')

#initial view

elev0, azim0 = 25, 30
ax.view_init(elev0, azim0)

#scatter objects

scn = ax.scatter([], [], [], s=0.5, alpha=0.1, c='cyan', edgecolors='none')
sch = ax.scatter([], [], [], s=[], alpha=0.7, c='gold', edgecolors='none')
txt = ax.text2D(0.02, 0.95, '', transform=ax.transAxes, color='white')
ax_triad = inset_axes(ax, width='20%', height='20%', loc='lower left')
ax_triad.axis('off')

def rotation_matrix(angle_deg):
    theta = np.deg2rad(angle_deg)
    Rz = np.array([[np.cos(theta), -np.sin(theta), 0],
                   [np.sin(theta),  np.cos(theta), 0],
                   [0, 0, 1]])
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(theta), -np.sin(theta)],
                   [0, np.sin(theta),  np.cos(theta)]])
    return Rz @ Rx

#update function

def update(frame):
    # frame data
    idx = frame if frame < motion else motion - 1
    data_n = frames_n[idx]
    data_h = frames_h[idx]
    zval = frames_z[idx]

    #update N-body
    scn._offsets3d = (data_n[:,0], data_n[:,1], data_n[:,2])

    #update halos
    if data_h.size:
        masses = data_h[:,0]
        coords = data_h[:,1:]
        radii = np.cbrt(masses)
        sizes = (radii / radii.max()) * 50
        sch._offsets3d = (coords[:,0], coords[:,1], coords[:,2])
        sch.set_sizes(sizes)
    else:
        sch._offsets3d = ([], [], [])
        sch.set_sizes([])

    #text
    txt.set_text(f'z = {zval:.3f}')

    #rotate view
    angle = frame * rotation_speed
    ax.view_init(elev0 + angle, azim0 + angle)

    #inset triad
    ax_triad.clear()
    ax_triad.axis('off')
    R = rotation_matrix(angle)
    for color, vec, label in zip(['red','green','blue'], [np.array([1,0,0]), np.array([0,1,0]), np.array([0,0,1])], ['X','Y','Z']):
        v2 = R @ vec
        end = np.array([0.5, 0.5]) + 0.3 * v2[:2]
        ax_triad.annotate('', xy=end, xytext=(0.5, 0.5), arrowprops=dict(color=color, arrowstyle='->', alpha=0.6))
        ax_triad.text(end[0], end[1], label, color=color, fontsize=8, alpha=0.6)

    #zoom
    if frame >= motion:
        h = frame - motion
        if h <= zoom_in:
            f = 1 + (max_zoom - 1) * (h / zoom_in)
        else:
            f = max_zoom - (max_zoom - 1) * min((h - zoom_in) / zoom_out, 1)
    else:
        f = 1
    for i, (mn,mx) in enumerate(bounds):
        c = centers[i]
        r = (mx - mn) / 2
        low, high = c - r/f, c + r/f
        if i == 0:
            ax.set_xlim(low, high)
        elif i == 1:
            ax.set_ylim(low, high)
        else:
            ax.set_zlim(low, high)

    return scn, sch, txt

#animate & save

ani = FuncAnimation(fig, update, frames=total_frames, interval=1000/fps, blit=True)
writer = FFMpegWriter(fps=fps, metadata={'artist':'PragunNepal'}, bitrate=30000)
ani.save('N-Body_Simulation_with_Halos_with_axes.mp4', writer=writer)
print('✅ Saved N-Body_Simulation_with_Halos_with_axes.mp4')
```
---

## 🔭 <a name="conclusion"></a>4. Conclusion

The code has been made general, hence its easily customisable. Please make necessary changes to the code as per your needs. 

©Pragun Nepal, 2025

Contact Me :
- Email : pragunnepal@gmail.com
- Email : pragun23@iisertvm.ac.in
- GitHub : PragunNepal
- LinkedIn : Pragun Nepal